<a href="https://colab.research.google.com/github/Lakshit1111/AI-Agents/blob/main/Content-Writer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install crewai crewai_tools litellm databricks-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.5/677.5 kB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 15.0 MB

In [2]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=125ba756703aa0c7a79b6f39a1eb03e2aa54f6482fcd13962cc909da0681b93b
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [3]:
!pip install langchain-community

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import os
os.environ["GROQ_API_KEY"] = ''
os.environ["SERPER_API_KEY"] = ""

#Part 1

In [ ]:
from crewai import Agent, Task, Crew, LLM

In [ ]:

Llm = LLM(
    api_key=os.getenv("GROQ_API_KEY"),
    model="groq/deepseek-r1-distill-qwen-32b",
    temperature=0.7,
    max_tokens = 2000
)

In [ ]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True,
    llm=Llm,
    max_rpm = 2
)

In [ ]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True,
    llm = Llm,
    max_rpm = 2
)

In [ ]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True,
    llm=Llm,
    max_rpm = 2
)

In [ ]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [ ]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [ ]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [ ]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose = True,
    max_rpm = 3
)

In [ ]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 
[2025-03-14 20:10:10][🚀 CREW 'CREW' STARTED, 9CBAE194-6FBD-4813-89C6-CED2167EE819]: 2025-03-14 20:10:10.129246
 
[2025-03-14 20:10:10][📋 TASK STARTED: 1. PRIORITIZE THE LATEST TRENDS, KEY PLAYERS, AND NOTEWORTHY NEWS ON ARTIFICIAL INTELLIGENCE.
2. IDENTIFY THE TARGET AUDIENCE, CONSIDERING THEIR INTERESTS AND PAIN POINTS.
3. DEVELOP A DETAILED CONTENT OUTLINE INCLUDING AN INTRODUCTION, KEY POINTS, AND A CALL TO ACTION.
4. INCLUDE SEO KEYWORDS AND RELEVANT DATA OR SOURCES.]: 2025-03-14 20:10:10.147064
 
[2025-03-14 20:10:10][🤖 AGENT 'CONTENT PLANNER' STARTED TASK]: 2025-03-14 20:10:10.148210
# Agent: Content Planner
## Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.
 
[2025-03-14 20:10:10][🤖 LLM CALL STARTED

In [ ]:
from IPython.display import Markdown
Markdown(result.raw)

**

# Artificial Intelligence 2023: Trends, Applications, and the Road Ahead

## Introduction

Artificial Intelligence (AI) has transcended its fictional origins to become an integral part of our daily lives. Beyond the surface-level conveniences, AI is revolutionizing industries, offering innovative solutions to complex problems. This article delves into the latest trends, applications, and ethical considerations of AI in 2023, offering a comprehensive view of this transformative technology.

## Latest Trends in AI

### Generative AI: A Creative Revolution
Generative AI is pioneering a new era of creativity, enabling the production of text, images, and music. Tools like ChatGPT and Midjourney are reshaping content creation, presenting both opportunities and challenges, such as copyright issues and deepfakes, necessitating cautious implementation.

### AI in Healthcare: Transforming Diagnosis and Treatment
AI's influence in healthcare is significant, enhancing diagnostic accuracy, predicting disease outbreaks, and aiding drug discovery. Personalized treatment plans are becoming a reality, promising improved patient outcomes and efficient healthcare delivery.

### AI in Business: Driving Efficiency and Insights
Businesses are harnessing AI for automation, improving efficiency and decision-making. Chatbots offer round-the-clock customer service, while predictive analytics provide market insights, empowering businesses to remain competitive.

### Ethical AI: Ensuring Fairness and Transparency
As AI becomes ubiquitous, ethical concerns loom large. Addressing issues like algorithmic bias and privacy is essential to ensure AI systems are fair and transparent, fostering public trust and inclusive growth.

## Key Players in AI

### Tech Titans at the Forefront
Google, Microsoft, Amazon, and Apple are driving AI innovation. Google's DeepMind and Microsoft's Azure AI lead advancements, while Amazon and Apple integrate AI into their ecosystems.

### Startups Innovating the Future
Startups like DeepMind, OpenAI, and Vicarious are pushing AI boundaries, exploring machine learning and autonomous systems, contributing significantly to AI's evolution.

### Research Institutions: The Pillars of Progress
Institutions like MIT, Stanford, and UC Berkeley are research hubs, producing studies that shape AI, crucial for overcoming challenges and realizing its potential.

## Applications of AI

### Consumer Applications: Enhancing Everyday Life
AI enhances daily life through smart homes, virtual assistants, and recommendation systems, offering convenience and personalization, making it an integral part of modern living.

### Industrial Applications: Transforming Production
AI revolutionizes industries via robotics, supply chain optimization, and predictive maintenance, boosting efficiency and reducing costs, driving industrial growth.

### Societal Impact: Addressing Global Challenges
AI contributes to solving global issues like climate change and education, providing data-driven solutions for a sustainable future.

## Ethical and Future Considerations

### Bias and Fairness: Building Trust in AI
Ensuring AI systems are unbiased is critical for public trust. Efforts to develop fair AI are essential for equitable growth.

### Job Market: Embracing Change and Opportunity
While AI disrupts traditional roles, it also creates new opportunities. Upskilling is vital to harness AI's benefits responsibly.

### Future of AI: Visions Beyond 2023
The future holds promise with advancements like AGI, yet responsible development through regulation and ethical frameworks is essential.

## Conclusion

AI is a powerful tool with transformative potential. Collaborative efforts among governments, businesses, and individuals are crucial to navigating AI's challenges and opportunities, paving the way for a brighter future.

## Call to Action

Explore AI's potential, stay informed, and engage with ethical implications. Together, we can shape a future where AI benefits all.

---

This revised version enhances clarity, balance, and adherence to the brand's voice, ensuring it meets all specified criteria for publication.

In [6]:
from crewai import Agent, Task, Crew, LLM
from langchain_community.tools.google_finance import GoogleFinanceQueryRun
from langchain_community.utilities.google_finance import GoogleFinanceAPIWrapper

In [7]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)
websiteScraper = ScrapeWebsiteTool(website_url='https://www.screener.in/screens/440753/price-volume-action/')
websiteScraper1 = ScrapeWebsiteTool()
search_tool = SerperDevTool(api_key=os.getenv("SERPAPI_API_KEY"))


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/selenium_scraping_tool/selenium_scraping_tool.py:26: PydanticDeprecatedSince20: Pydantic V1 style `@validator`

In [8]:
Llm = LLM(
    api_key=os.getenv("GROQ_API_KEY"),
    model="groq/deepseek-r1-distill-qwen-32b",
    temperature=0.7,
    max_tokens = 2000
)

In [ ]:
companies_getter = Agent(
    role = "Corporate Research Specialist ",
    goal = "Get names of companies from the given site",
    backstory = "You are a Senior Data Collector for 16 Years"
                "You find companies having growth potential"
                "You do not assume data"
                "You work on the basis of new data",
    tools = [websiteScraper],
    llm = Llm,
    max_rpm=2
)

In [ ]:
companies_information_Getter = Agent(
    role = "Company Research Analyst",
    goal = "Get information about companies from the companies_getter agent",
    backstory = "Five years in market research."
                "Degree in Business Management."
                "Skilled in data analysis."
                "Supports senior analysts effectively."
                "Committed to company success",
    tools = [search_tool , websiteScraper1],
    llm = Llm,
    max_rpm=2
)

In [ ]:
companies_Getter_task = Task(
    description =
      "Get Names of the company form the website",
    expected_output =
      "Name of comapany with their NSE symbol"
      "Store that data in cache so that other agent can access them.",
    agent =  companies_getter
)

In [ ]:
companies_information_Getter_task = Task(
    description =
      "Get information about companies from the companies_getter agent from cache"
      "Use the tool"
      "Conduct research from internet to identify and collect"
      "informataion on given companies."
      "Information should be related to growth potential"
      "of the company and how its outlook in near future."
      "Research should be according to present time.",
    expected_output =
      "Brief of company working and how it earn profit"
      "monthly and yearly outlook of company.",
    agent =  companies_information_Getter
)

In [ ]:
companies_information_Getter_task = Task(
    description =
      "Get companies from the companies_getter agent from cache"
      "Give me latest news article"
      "Do not assume data"
      "Get latest data",
    expected_output =
      "Latest news articles",
    agent =  companies_information_Getter,
)

In [ ]:
crew = Crew(
    agents=[companies_getter, companies_information_Getter],
    tasks=[companies_Getter_task , companies_information_Getter_task],
    verbose = True,
    cache = True
)

In [ ]:
result = crew.kickoff()

 
[2025-03-16 11:17:30][🚀 CREW 'CREW' STARTED, AB823000-324D-49D1-BA3D-17B36D66F5E7]: 2025-03-16 11:17:30.448130
 
[2025-03-16 11:17:30][📋 TASK STARTED: GET NAMES OF THE COMPANY FORM THE WEBSITE]: 2025-03-16 11:17:30.462227
 
[2025-03-16 11:17:30][🤖 AGENT 'CORPORATE RESEARCH SPECIALIST ' STARTED TASK]: 2025-03-16 11:17:30.468795
# Agent: Corporate Research Specialist 
## Task: Get Names of the company form the website
 
[2025-03-16 11:18:30][🤖 LLM CALL STARTED]: 2025-03-16 11:18:30.469349
 
[2025-03-16 11:18:34][✅ LLM CALL COMPLETED]: 2025-03-16 11:18:34.383615
 
[2025-03-16 11:18:34][🤖 TOOL USAGE STARTED: 'READ WEBSITE CONTENT']: 2025-03-16 11:18:34.384197
 
[2025-03-16 11:18:35][✅ TOOL USAGE FINISHED: 'READ WEBSITE CONTENT']: 2025-03-16 11:18:35.485710


# Agent: Corporate Research Specialist 
## Using tool: Read website content
## Tool Input: 
"{\"url\": \"https://www.screener.in/screens/440753/price-volume-action/\"}"
## Tool Output: 

Price Volume Action - Screener
Home
Screens
To

In [11]:
import os
from crewai_tools import SerperDevTool
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
serper_api_key = os.getenv("SERPER_API_KEY")

# Initialize the search tool
search_tool = SerperDevTool()

# Define the search query
search_query = "latest trends in artificial intelligence"

# Run the search
try:
    # Calling the run method with a dictionary as a single argument
    response = search_tool.run()

    # Check and print the result
    print(response)  # Make sure to handle the response as per your needs
except Exception as e:
    print(f"An error occurred: {e}")


ERROR:crewai_tools.tools.serper_dev_tool.serper_dev_tool:Error making request to Serper API: 403 Client Error: Forbidden for url: https://google.serper.dev/search
Response content: b'{"message":"Unauthorized.","statusCode":403}'


Using Tool: Search the internet with Serper
An error occurred: 403 Client Error: Forbidden for url: https://google.serper.dev/search
